In [2]:
import src
import keras.backend as K
import os
import numpy as np
import sys
import re
import math
import io
from scipy import ndimage
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from  matplotlib.animation import FuncAnimation
from matplotlib import colors
from netCDF4 import Dataset
from IPython.display import clear_output
#data folder
sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/ADL4CV_project/trainings')

sys.path.insert(0, 'C:/Users/pkicsiny/Desktop/TUM/3/ADL4CV/data')
#forces CPU usage
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0" #"" or "-1" for CPU, "0" for GPU
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 985959920689696173
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1508248780
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18342388926032076029
physical_device_desc: "device: 0, name: GeForce GT 740M, pci bus id: 0000:01:00.0, compute capability: 3.5"
]


In [3]:
def plot_training_curves(log, epoch, name):
    total_g_loss = np.array(log["g_loss"])[:, 0]
    total_ds_loss = np.array(log["ds_loss"])
    total_dt_loss = np.array(log["dt_loss"])
    smoothed_tgl = src.smooth(np.array(log["g_loss"])[:, 0])
    smoothed_tdsl = src.smooth(np.array(log["ds_loss"]))
    smoothed_tdtl = src.smooth(np.array(log["dt_loss"]))
    objective_loss = np.array(log["g_loss"])[:, 1]

    f, (a0, a1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [5, 2]})
    a0.plot(total_g_loss, alpha=0.3, c="b")
    a0.plot(total_ds_loss, alpha=0.3, c="orange")
    a0.plot(total_dt_loss, alpha=0.3, c="red")
    a0.plot(smoothed_tgl, c="b", label="G")
    a0.grid()
    a0.plot(smoothed_tdsl, c="orange", label="Ds")
    a0.plot(smoothed_tdtl, c="red", label="Dt")
    a0.legend()
    a1.plot(objective_loss, alpha=0.9, c="green", label="L1 objective")
    a1.grid()
    a1.legend()
    f.text(0.5, 0, 'Iterations', ha='center', va='center')
    f.text(0, 0.5, 'Loss', ha='center', va='center', rotation='vertical')

    f.tight_layout()
    f.savefig(f"Plots/{name}_epoch_{epoch}_curves.png")

def plot_advections(advected_aux_gen, advected_aux_truth):
    gen = advected_aux_gen[:5]
    truth =  advected_aux_truth[:5]
    fig, axs = plt.subplots(5, 2, figsize=(16, 16))
    for i in range(5):
        vmax = np.max([np.max(gen[i]), np.max(truth[i])])
        vmin = 0
        im = axs[i,0].imshow(gen[i, :,:,0], vmax=vmax,vmin=vmin)
        axs[i,0].axis('off')
        src.colorbar(im)
        axs[i,0].set_title("Advected generated frame")
        
        im = axs[i,1].imshow(truth[i, :,:,0], vmax=vmax,vmin=vmin)
        axs[i,1].axis('off')
        src.colorbar(im)
        axs[i,1].set_title("Advected reference frame")
        
    fig.savefig("Plots/advections_epoch %d.png" % epoch)
    plt.close()

In [4]:
past = 2
name = f"tgan_{past}-1"

Load dataset.

In [5]:
train, xval, test = src.load_datasets(past_frames=past+1)

Training data: (7500, 64, 64, 4)
Validation data: (1500, 64, 64, 4)
Test data: (1000, 64, 64, 4)


Split data to inputs and ground truth images.

In [6]:
gan_train, gan_truth, gan_val, gan_val_truth, gan_test, gan_test_truth = src.split_datasets(
            train[:2000], xval, test, past_frames=past+1, augment=True)

Data augmentation.
Shape of training data:  (8000, 64, 64, 3) 
Shape of training truth:  (8000, 64, 64, 1) 
Shape of validation data:  (1500, 64, 64, 3) 
Shape of validation truth:  (1500, 64, 64, 1) 
Shape of test data:  (1000, 64, 64, 3) 
Shape of test truth:  (1000, 64, 64, 1)


Calculate optical flows between frame t-1 and t.

In [ ]:
#optical flow of the augmented data of the first 2000 training images (8000 images)
#vx, vy = src.optical_flow(gan_train[:,:,:,-2:-1], gan_train[:,:,:,-1:], window_size=4, tau=1e-2, init=1) # (n,:,:,1)

Save optical flows

In [ ]:
#np.savez_compressed(f"{name}_vx_2000",vx) #2000 denotes that they re the flo2 of the first 2000 samples from the training dataset
#np.savez_compressed(f"{name}_vy_2000",vy)

If optical flows are saved, load them

In [7]:
vx = np.load(sys.path[0]+f"/{name}_vx_2000.npz")["arr_0"]
vy = np.load(sys.path[0]+f"/{name}_vy_2000.npz")["arr_0"]

Normalize flows

In [8]:
normalized_optical_flow = src.normalize_flows(vx, vy)

Smooth flows.

In [9]:
median = np.transpose([[ndimage.median_filter(image[...,ch], 4) for ch in range(2)] for image in normalized_optical_flow], (0,2,3,1))

Make discriminator labels.

In [98]:
batch_size=64

In [113]:
# Adversarial ground truths
real = -np.ones((batch_size, 1))
fake = np.ones((batch_size, 1))

Switch 5% of labels and smooth from 1 to 0.9.

In [114]:
#real, fake = src.noisy_d_labels(real, fake)

Make generator but don't compile.

In [127]:
generator = src.unet((64, 64, past), dropout=0, batchnorm=True, kernel_size=4, feature_mult=1)

Make discriminators and compile.

In [128]:
s_discriminator = src.spatial_discriminator(condition_shape=(64, 64, past), dropout = 0.25, batchnorm=True, wgan=True)
s_discriminator.compile(loss=src.wasserstein_loss,
                        optimizer=keras.optimizers.RMSprop(lr=0.00005))

In [129]:
t_discriminator = src.temporal_discriminator(dropout = 0.25, batchnorm=True, wgan=True)
t_discriminator.compile(loss=src.wasserstein_loss,
                        optimizer=keras.optimizers.RMSprop(lr=0.00005))

Inputs and outputs of the GAN.

In [130]:
frame_t = keras.layers.Input(shape=(64, 64, past))
adv = keras.layers.Input(shape=(64, 64, 1))

In [131]:
generated = generator(frame_t)
s_score = s_discriminator([frame_t, generated])
t_score = t_discriminator([adv, generated])

Freeze discriminator weights.

In [132]:
s_discriminator.trainable = False
t_discriminator.trainable = False

Compile combined model.

In [133]:
loss_weights=[0, 1, 1]

In [134]:
combined = keras.models.Model(inputs=[frame_t, adv], outputs=[generated, s_score, t_score])

In [135]:
combined.compile(loss=[src.custom_loss(loss="l1"),
                       src.wasserstein_loss,
                       src.wasserstein_loss],
                 optimizer=keras.optimizers.RMSprop(lr=0.00005),
                 loss_weights=loss_weights)

***Using L1 loss.***


Train x epochs.

In [136]:
iterations = 5000
g = 1
nb_batches = int(gan_train.shape[0] / batch_size)

In [137]:
log = {"g_loss":[],
       "d_loss":[],
       "ds_loss":[],
       "dt_loss":[],
       "g_metric":[],
       "d_metric":[],
       "ds_metric":[],
       "dt_metric":[],
       "d_loss_real":[],
       "d_loss_fake":[],
       "ds_loss_real":[],
       "ds_loss_fake":[],
       "dt_loss_real":[],
       "dt_loss_fake":[],}

In [138]:
for it in range(iterations):
#batch
#idx = range(it%nb_batches * batch_size,(it%nb_batches + 1) * batch_size)
#DS create training batch
    if (it % 1000) < 25 or it % 500 == 0: # 25 times in 1000, every 500th
        d_iters = 10
    else:
        d_iters = 10
    for d_it in range(d_iters):
        s_discriminator.trainable = True
        for l in s_discriminator.layers: l.trainable = True
        idx = np.random.randint(0, gan_truth.shape[0], batch_size)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx,:,:,1:]
        
        for l in s_discriminator.layers:
            weights = l.get_weights()
            weights = [np.clip(w, -0.01, 0.01) for w in weights]
            l.set_weights(weights)
        
        #predict next frame  
        generated_imgs = generator.predict(training_batch) 
        #train discriminator
        ds_loss_real = s_discriminator.train_on_batch([training_batch, real_imgs], real)
        ds_loss_fake = s_discriminator.train_on_batch([training_batch, generated_imgs], fake)
        #add losses
        ds_loss = 0.5 * np.add(ds_loss_real, ds_loss_fake)
        s_discriminator.trainable = False
        for l in s_discriminator.layers: l.trainable = False
    
    #DT create training batch
        t_discriminator.trainable = True
        for l in t_discriminator.layers: l.trainable = True
        idx = np.random.randint(0, gan_truth.shape[0], batch_size)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx,:,:,1:]
        aux_batch = gan_train[idx,:,:,:-1]
        
        for l in t_discriminator.layers:
            weights = l.get_weights()
            weights = [np.clip(w, -0.01, 0.01) for w in weights]
            l.set_weights(weights)
            
        aux_gen_imgs = generator.predict(aux_batch)
        aux_gen_imgs = np.concatenate((aux_gen_imgs, median[idx]), axis=-1)
        advected_aux_gen = np.array([src.advect(sample) for sample in aux_gen_imgs])
        #advect real frame t
        aux_true_imgs = training_batch[:,:,:,-1:]
        aux_true_imgs = np.concatenate((aux_true_imgs, median[idx]), axis=-1)
        advected_aux_truth = np.array([src.advect(sample) for sample in aux_true_imgs])
        #predict next frame
        generated_imgs = generator.predict(training_batch) 
        #train discriminator
        dt_loss_real = t_discriminator.train_on_batch([advected_aux_truth, real_imgs], real)
        dt_loss_fake = t_discriminator.train_on_batch([advected_aux_gen, generated_imgs], fake)
        #add losses
        dt_loss = 0.5 * np.add(dt_loss_real, dt_loss_fake)
        t_discriminator.trainable = False
        for l in t_discriminator.layers: l.trainable = False

    #Total discriminator loss
        d_loss = 0.5 * np.add(ds_loss, dt_loss)
        d_loss_real = 0.5 * np.add(ds_loss_real, dt_loss_real)
        d_loss_fake = 0.5 * np.add(ds_loss_fake, dt_loss_fake)
        
        print(f"\033[1m {it}/{d_it} [D loss real: {d_loss_real}\n    D loss fake: {d_loss_fake}] \n"+
              f"\033[1m {it}/{d_it} [Ds loss real: {ds_loss_real}\n    Ds loss fake: {ds_loss_fake}] \033[0m \n"+
              f"\033[1m {it}/{d_it} [t loss real: {dt_loss_real}\n    Dt loss fake: {dt_loss_fake}] \033[0m")

#Generator
    for tg in range(g):
        idx = np.random.randint(0, gan_train.shape[0], batch_size)
        real_imgs = gan_truth[idx]
        training_batch = gan_train[idx,:,:,1:]
        
        aux_batch = gan_train[idx,:,:,:-1]
        aux_gen_imgs = generator.predict(aux_batch)
        aux_gen_imgs = np.concatenate((aux_gen_imgs, median[idx]), axis=-1)
        advected = np.array([src.advect(sample) for sample in aux_gen_imgs])
    
        g_loss = combined.train_on_batch([training_batch, advected], [real_imgs, real, real])
    
    log["g_loss"].append(g_loss)
    log["d_loss"].append(d_loss)
    log["ds_loss"].append(ds_loss) 
    log["dt_loss"].append(dt_loss)
    log["d_loss_real"].append(d_loss_real)
    log["d_loss_fake"].append(d_loss_fake)
    log["ds_loss_real"].append(ds_loss_real)
    log["ds_loss_fake"].append(ds_loss_fake)
    log["dt_loss_real"].append(dt_loss_real)
    log["dt_loss_fake"].append(dt_loss_fake)
    
    
    print(f"\033[1m {it} [Ds loss: {ds_loss}]\033[0m \n"+
          f"\033[1m {it} [Dt loss: {dt_loss}]\033[0m \n"+
          f"\033[1m {it} [G loss: {g_loss}]\033[0m \n"+
          f"S: real loss: {ds_loss_real}, fake loss: {ds_loss_fake} \n"+
          f"T: real loss: {dt_loss_real}, fake loss: {dt_loss_fake}")
    if it%100 == 0 and it>0:
        src.sample_images(it, gan_test[...,1:], gan_test_truth, past, generator)
        plot_training_curves(log, it, name)

src.sample_images(iterations, gan_test[...,1:], gan_test_truth, past, generator)
plot_training_curves(log, iterations, name)

 0/0 [D loss real: 1.0927341520527989e-07
    D loss fake: 0.0005333640147000551] 
 0/0 [Ds loss real: -2.8180771494135115e-08
    Ds loss fake: 0.0005801617517136037]  
 0/0 [t loss real: 2.4672760901012225e-07
    Dt loss fake: 0.0004865662194788456] 
 0/1 [D loss real: -0.0001178117745439522
    D loss fake: 0.000429862440796569] 
 0/1 [Ds loss real: -0.00012097350554540753
    Ds loss fake: 0.0004615611396729946]  
 0/1 [t loss real: -0.00011465004354249686
    Dt loss fake: 0.00039816374192014337] 
 0/2 [D loss real: -0.00014469382585957646
    D loss fake: 0.0003857076517306268] 
 0/2 [Ds loss real: -0.00015134467685129493
    Ds loss fake: 0.00041432451689615846]  
 0/2 [t loss real: -0.00013804296031594276
    Dt loss fake: 0.0003570907865650952] 
 0/3 [D loss real: -0.00015551831165794283
    D loss fake: 0.0003656072949524969] 
 0/3 [Ds loss real: -0.0001644622825551778
    Ds loss fake: 0.00039617420407012105]  
 0/3 [t loss real: -0.00014657434076070786
    Dt loss fake: 0.

 2/8 [D loss real: -0.00012624790542759
    D loss fake: 0.00024583653430454433] 
 2/8 [Ds loss real: -0.00012988824164494872
    Ds loss fake: 0.0002634574775584042]  
 2/8 [t loss real: -0.00012260755465831608
    Dt loss fake: 0.00022821559105068445] 
 2/9 [D loss real: -0.0001262873411178589
    D loss fake: 0.00024328914878424257] 
 2/9 [Ds loss real: -0.00012933241669088602
    Ds loss fake: 0.0002556905965320766]  
 2/9 [t loss real: -0.00012324226554483175
    Dt loss fake: 0.00023088770103640854] 
 2 [Ds loss: 6.317908992059529e-05] 
 2 [Dt loss: 5.3822717745788395e-05] 
 2 [G loss: [-0.0002478129, 0.39318973, -0.00012655783, -0.00012125507]] 
S: real loss: -0.00012933241669088602, fake loss: 0.0002556905965320766 
T: real loss: -0.00012324226554483175, fake loss: 0.00023088770103640854
 3/0 [D loss real: -0.00012315584172029048
    D loss fake: 0.00023876968771219254] 
 3/0 [Ds loss real: -0.00012236199108883739
    Ds loss fake: 0.00024891505017876625]  
 3/0 [t loss real: -

 5/5 [D loss real: -9.341069380752742e-05
    D loss fake: 0.00018966387142427266] 
 5/5 [Ds loss real: -9.228716953657568e-05
    Ds loss fake: 0.00019628451263997704]  
 5/5 [t loss real: -9.453421080252156e-05
    Dt loss fake: 0.0001830432447604835] 
 5/6 [D loss real: -9.368440078105778e-05
    D loss fake: 0.00018844968872144818] 
 5/6 [Ds loss real: -9.178047184832394e-05
    Ds loss fake: 0.00019867926312144846]  
 5/6 [t loss real: -9.558832971379161e-05
    Dt loss fake: 0.0001782201143214479] 
 5/7 [D loss real: -9.15085693122819e-05
    D loss fake: 0.00018520059529691935] 
 5/7 [Ds loss real: -9.33728224481456e-05
    Ds loss fake: 0.0001928234560182318]  
 5/7 [t loss real: -8.964432345237583e-05
    Dt loss fake: 0.0001775777491275221] 
 5/8 [D loss real: -9.220508218277246e-05
    D loss fake: 0.00018316871137358248] 
 5/8 [Ds loss real: -9.15591954253614e-05
    Ds loss fake: 0.00019030530529562384]  
 5/8 [t loss real: -9.285097621614113e-05
    Dt loss fake: 0.000176

 8/2 [D loss real: -7.63038988225162e-05
    D loss fake: 0.00014881734387017787] 
 8/2 [Ds loss real: -7.4179915827699e-05
    Ds loss fake: 0.00014643906615674496]  
 8/2 [t loss real: -7.84278818173334e-05
    Dt loss fake: 0.000151195636135526] 
 8/3 [D loss real: -7.519533392041922e-05
    D loss fake: 0.00014735109289176762] 
 8/3 [Ds loss real: -7.30763713363558e-05
    Ds loss fake: 0.0001460761413909495]  
 8/3 [t loss real: -7.731428922852501e-05
    Dt loss fake: 0.00014862604439258575] 
 8/4 [D loss real: -7.328046194743365e-05
    D loss fake: 0.00014630399527959526] 
 8/4 [Ds loss real: -7.097296474967152e-05
    Ds loss fake: 0.00014342233771458268]  
 8/4 [t loss real: -7.558795186923817e-05
    Dt loss fake: 0.00014918565284460783] 
 8/5 [D loss real: -7.398611342068762e-05
    D loss fake: 0.00014197040582075715] 
 8/5 [Ds loss real: -7.030670531094074e-05
    Ds loss fake: 0.00013905971718486398]  
 8/5 [t loss real: -7.766552153043449e-05
    Dt loss fake: 0.0001448

 10 [Ds loss: 3.491027018753812e-05] 
 10 [Dt loss: 2.8580467187566683e-05] 
 10 [G loss: [-0.00013144157, 0.40427667, -6.461471e-05, -6.682687e-05]] 
S: real loss: -6.987534288782626e-05, fake loss: 0.0001396958832629025 
T: real loss: -6.923436740180477e-05, fake loss: 0.00012639530177693814
 11/0 [D loss real: -6.480887532234192e-05
    D loss fake: 0.00013163895346224308] 
 11/0 [Ds loss real: -6.579369073733687e-05
    Ds loss fake: 0.0001310480001848191]  
 11/0 [t loss real: -6.382405263138935e-05
    Dt loss fake: 0.00013222990673966706] 
 11/1 [D loss real: -6.717450742144138e-05
    D loss fake: 0.0001324343029409647] 
 11/1 [Ds loss real: -6.424466118915007e-05
    Ds loss fake: 0.00013074598973616958]  
 11/1 [t loss real: -7.01043609296903e-05
    Dt loss fake: 0.00013412261614575982] 
 11/2 [D loss real: -6.677969940938056e-05
    D loss fake: 0.00012834189692512155] 
 11/2 [Ds loss real: -6.540778849739581e-05
    Ds loss fake: 0.00012853837688453496]  
 11/2 [t loss rea

...

 19/1 [D loss real: -6.67167769279331e-05
    D loss fake: 0.00012588745448738337] 
 19/1 [Ds loss real: -6.657280027866364e-05
    Ds loss fake: 0.00012276327470317483]  
 19/1 [t loss real: -6.686075357720256e-05
    Dt loss fake: 0.00012901161971967667] 
 19/2 [D loss real: -7.0329406298697e-05
    D loss fake: 0.00012193401926197112] 
 19/2 [Ds loss real: -7.051556167425588e-05
    Ds loss fake: 0.00012253783643245697]  
 19/2 [t loss real: -7.014325819909573e-05
    Dt loss fake: 0.00012133020209148526] 
 19/3 [D loss real: -6.586447125300765e-05
    D loss fake: 0.0001251925714313984] 
 19/3 [Ds loss real: -6.784356810385361e-05
    Ds loss fake: 0.0001253507361980155]  
 19/3 [t loss real: -6.38853816781193e-05
    Dt loss fake: 0.00012503439211286604] 
 19/4 [D loss real: -7.072043081279844e-05
    D loss fake: 0.00012260410585440695] 
 19/4 [Ds loss real: -6.707714055664837e-05
    Ds loss fake: 0.0001231836504302919]  
 19/4 [t loss real: -7.436372106894851e-05
    Dt loss fa

 21/9 [D loss real: -6.518174632219598e-05
    D loss fake: 0.00012320990208536386] 
 21/9 [Ds loss real: -6.511858373414725e-05
    Ds loss fake: 0.0001228353357873857]  
 21/9 [t loss real: -6.52449089102447e-05
    Dt loss fake: 0.00012358446838334203] 
 21 [Ds loss: 2.8858376026619226e-05] 
 21 [Dt loss: 2.9169779736548662e-05] 
 21 [G loss: [-0.00012856998, 0.40052265, -6.4119275e-05, -6.445071e-05]] 
S: real loss: -6.511858373414725e-05, fake loss: 0.0001228353357873857 
T: real loss: -6.52449089102447e-05, fake loss: 0.00012358446838334203
 22/0 [D loss real: -6.181482604006305e-05
    D loss fake: 0.00012507467181421816] 
 22/0 [Ds loss real: -6.36247787042521e-05
    Ds loss fake: 0.00012505946506280452]  
 22/0 [t loss real: -6.000487337587401e-05
    Dt loss fake: 0.00012508986401371658] 
 22/1 [D loss real: -6.659759674221277e-05
    D loss fake: 0.0001227409957209602] 
 22/1 [Ds loss real: -6.556307926075533e-05
    Ds loss fake: 0.00012421875726431608]  
 22/1 [t loss rea

 24/6 [D loss real: -6.780271360184997e-05
    D loss fake: 0.0001258832198800519] 
 24/6 [Ds loss real: -6.893699173815548e-05
    Ds loss fake: 0.00012565548240672797]  
 24/6 [t loss real: -6.666843546554446e-05
    Dt loss fake: 0.00012611095735337585] 
 24/7 [D loss real: -6.663009116891772e-05
    D loss fake: 0.0001197231758851558] 
 24/7 [Ds loss real: -6.944938650121912e-05
    Ds loss fake: 0.0001214350777445361]  
 24/7 [t loss real: -6.38107885606587e-05
    Dt loss fake: 0.00011801127402577549] 
 24/8 [D loss real: -6.78774667903781e-05
    D loss fake: 0.00012431427603587508] 
 24/8 [Ds loss real: -6.891050725243986e-05
    Ds loss fake: 0.00012554015847854316]  
 24/8 [t loss real: -6.684442632831633e-05
    Dt loss fake: 0.00012308840814512223] 
 24/9 [D loss real: -6.660989311058074e-05
    D loss fake: 0.0001238819386344403] 
 24/9 [Ds loss real: -7.149734301492572e-05
    Ds loss fake: 0.00012208404950797558]  
 24/9 [t loss real: -6.172244320623577e-05
    Dt loss f

 27/3 [D loss real: -7.013229333097115e-05
    D loss fake: 0.0001233900838997215] 
 27/3 [Ds loss real: -7.266849570441991e-05
    Ds loss fake: 0.00012421287829056382]  
 27/3 [t loss real: -6.759609095752239e-05
    Dt loss fake: 0.0001225673040607944] 
 27/4 [D loss real: -7.322562305489555e-05
    D loss fake: 0.0001219093392137438] 
 27/4 [Ds loss real: -7.505432586185634e-05
    Ds loss fake: 0.00011901272227987647]  
 27/4 [t loss real: -7.139692024793476e-05
    Dt loss fake: 0.00012480595614761114] 
 27/5 [D loss real: -7.277902477653697e-05
    D loss fake: 0.0001251674839295447] 
 27/5 [Ds loss real: -7.599891978316009e-05
    Ds loss fake: 0.00012502854224294424]  
 27/5 [t loss real: -6.955912976991385e-05
    Dt loss fake: 0.00012530642561614513] 
 27/6 [D loss real: -7.711408397881314e-05
    D loss fake: 0.00012393249198794365] 
 27/6 [Ds loss real: -8.556083776056767e-05
    Ds loss fake: 0.00012452837836463004]  
 27/6 [t loss real: -6.866733019705862e-05
    Dt loss

KeyboardInterrupt: 

In [ ]:
total_g_loss = np.array(log["g_loss"])[:,0]
total_d_loss = np.array(log["d_loss"])[:,0]
smoothed_tgl = src.smooth(np.array(log["g_loss"])[:,0])
smoothed_tdl = src.smooth(np.array(log["d_loss"])[:,0])
objective_loss = np.array(log["g_loss"])[:,1]

# plot 'em
f, (a0, a1) = plt.subplots(2,1, gridspec_kw = {'height_ratios':[5, 2]})
a0.plot(total_g_loss, alpha=0.3, c="b")
a0.plot(total_d_loss, alpha=0.3, c="orange")
a0.plot(smoothed_tgl, c="b", label="generator")
a0.grid()
a0.plot(smoothed_tdl, c="orange", label="discriminator")
a0.legend()
a1.plot(objective_loss, alpha=0.9, c="green", label="L1 objective")
a1.grid()
a1.legend()
f.text(0.5, 0, 'Iterations', ha='center', va='center')
f.text(0, 0.5, 'Loss', ha='center', va='center', rotation='vertical')

f.tight_layout()
f.savefig(name+'_curves.png')

## Save features

Save model history

In [ ]:
np.save(name+"_log",log)

Save model weights

In [ ]:
combined.save_weights(name+"_model.h5")

Load model and predict

In [ ]:
combined.load_weights(sys.path[1]+"/"+name+"/"+name+"_model.h5")

Predict future frames. Loads a 20 long sequence with 1000 sequence samples.

In [ ]:
sequence_test = src.load_datasets(prediction=True)

In [ ]:
sequence_test = src.augment_data(sequence_test[:100])

In [ ]:
sequence_test.shape

In [ ]:
generator = combined.layers[1]

In [ ]:
#test
predictions = {}
past_frames = sequence_test[...,0:past]
test_truth = sequence_test[...,past:past+1]
for t in range(5):
    future = generator.predict(past_frames, batch_size=64)
    predictions[f"{t}"] = future
    past_frames = np.concatenate((past_frames[:,:,:,1:], predictions[f"{t}"]), axis=-1)
    test_truth = sequence_test[...,past+1+t:past+2+t]

Save example predictions

In [ ]:
def save_examples(name, test, predictions_dict, past, samples=0):
    fig, axs = plt.subplots(len(samples)*2,past+len(predictions_dict.keys()), figsize=(32, 32))
    fig.subplots_adjust(wspace=0.3, hspace=0.0)
    for n in range(len(samples)):
        vmax = np.max(test[n,:,:,:past])
        vmin = 0
        print(test.shape)
        for i in range(past):
            im = axs[2*n,i].imshow(test[samples[n], :,:,i], vmax=vmax,vmin=vmin)
            axs[2*n,i].axis('off')
            axs[2*n,i].set_title(f"Past frame {i+1}")
            src.colorbar(im)
            im = axs[2*n+1,i].imshow(test[samples[n], :,:,i], vmax=vmax,vmin=vmin)
            axs[2*n+1,i].axis('off')
            axs[2*n+1,i].set_title(f"Past frame {i+1}")
            src.colorbar(im)
        for i in range(past,past+len(predictions_dict.keys())):
            im = axs[2*n,i].imshow(predictions_dict[f"{i-past}"][samples[n], :,:,0], vmax=vmax, vmin=vmin)
            axs[2*n,i].axis('off')
            axs[2*n,i].set_title(f"Predicted frame {i-past+1}")
            src.colorbar(im)
            im = axs[2*n+1,i].imshow(test[samples[n], :,:,i], vmax=vmax, vmin=vmin)
            axs[2*n+1,i].axis('off')
            axs[2*n+1,i].set_title(f"Reference frame {i-past+1}")
            src.colorbar(im)
    fig.savefig(f"Plots/{name}_sequence_prediction.png")
    plt.close()


In [ ]:
save_examples(name, sequence_test, predictions, past, samples=[33])

Renormalize intensity values

In [ ]:
norms = np.load(sys.path[0]+"/5min_norms_compressed.npz")["arr_1"]

In [ ]:
# *4 bc of augmentaion (it concats the frames so the 0th 1000th 2000th and 3000th are the same sample just rotated)
test_norms = list(norms[9000:])*4

In [ ]:
#renormalize test samples
renormalized_test = np.array([sample * np.array(test_norms)[i] for i, sample in enumerate(sequence_test)])
renormalized_predictions = np.transpose((np.array([[sample * np.array(test_norms)[i] for i, sample in enumerate(predictions[key])] for key in ['0', '1', '2', '3']])[:,:,:,:,0]), (1,2,3,0))

Calculate pixel intensities back to dBZ and from there to mm/h. <br>
Sources: <br>
- https://www.dwd.de/DE/leistungen/radolan/radolan_info/radolan_radvor_op_komposit_format_pdf.pdf?__blob=publicationFile&v=11 (page 10)
- <https://web.archive.org/web/20160113151652/http://www.desktopdoppler.com/help/nws-nexrad.htm#rainfall%20rates>

In [ ]:
#dBZ
dBZ_t = renormalized_test*0.5 - 32.5
dBZ_p = renormalized_predictions*0.5 - 32.5
#mm/h
I_t = (0.005*10**(0.1*dBZ_t))**(0.625)
I_p = (0.005*10**(0.1*dBZ_p))**(0.625)

In [ ]:
#thresholds: 2, 8, 42
thresholds = [0.5]
scores = {}
for t in range(I_t.shape[-1]): # loop over the predictions (4)
    for s in thresholds: # make a dict entry for each threshold score
        scores[f"pred_{t+1}"] = src.calculate_skill_scores(I_p[...,t:t+1],
                                                          I_t[...,past+t:past+1+t],
                                                          x=I_t[...,past-1:past],
                                                          threshold=s)

In [ ]:
scores["pred_1"].keys()

In [ ]:
np.save(name+"_scores",scores)

In [ ]:
name

In [ ]:
loaded_scores = np.load(sys.path[1]+"/"+name"/"+name+"_scores.npy").item()

In [ ]:
list(loaded_scores.keys())

In [ ]:
np.mean((pd.Series(scores["pred_2"]["corr_to_truth"]).dropna()))

In [ ]:
np.mean((pd.Series(scores["pred_2"]["corr_to_input"]).dropna()))

In [ ]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
layer_name = 'block5_conv3'
filter_index = 0  # can be any integer from 0 to 511, as there are 512 filters in that layer

# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_dict[layer_name].output
K.mean(layer_output[:, :, :, filter_index])